# Experiments of our custom types in SymPy.

Check if MathJax works: $E=mc^2$, $\Gamma\vdash x\colon \tau$

In [95]:
from sympy import S, init_printing, srepr, symbols, Symbol, Add, Function, Tuple, ITE, Piecewise, Lambda, Subs, simplify, sin
from sympy.core import Basic, sympify
from sympy.core.kind import Kind, NumberKind, BooleanKind
from sympy.core.expr import Expr
from sympy.core.symbol import Str

from pycheck.codegen.sympy_lib import ListKind, List, ListSymbol, Cons, IsNil, Len, Map, All, TupleSymbol
from pycheck.random.random_generators import rand_bool, rand_int

In [2]:
init_printing()

----
## Normal SymPy Expression

Case for normal addition

In [3]:
e1 = S('x+y+3')

In [4]:
e1

x + y + 3

In [5]:
(e1.func, e1.args)

(sympy.core.add.Add, (3, x, y))

In [6]:
srepr(e1)

"Add(Symbol('x'), Symbol('y'), Integer(3))"

In [7]:
e1.free_symbols

{x, y}

In [8]:
e1.subs('x', 2) # partially evaluated and simplified

y + 5

In [9]:
e1.subs('x', 2).subs('y', 4) # totally evaluated to an interger

9

In [10]:
srepr(e1.subs('x', 2).subs('y', 4))

'Integer(9)'

----
## Lists

### ListSymbol

In [11]:
a, b, x, y = symbols('a b x y')
l = ListSymbol('l')
m = ListSymbol('m')
f = Function('f')
g = Function('g')

In [12]:
(a, l, m)

(a, l, m)

In [13]:
str((a, l, m))

'(a, l, m)'

In [14]:
repr((a, l, m))

'(a, l, m)'

In [15]:
srepr((a, l, m))

"(Symbol('a'), ListSymbol(Str('l')), ListSymbol(Str('m')))"

In [16]:
(a.kind, l.kind)

(NumberKind, ListKind)

In [17]:
l.free_symbols

{l}

### Cons

In [18]:
Cons(a, m)

Cons(a, m)

In [19]:
Cons(a, m).func

pycheck.codegen.sympy_lib.Cons

In [20]:
(Cons(a,m).args, Cons(a,m).free_symbols)

((a, m), {a, m})

In [21]:
assert Cons(a, m).subs(a, 1) == Cons(1, m)

(Cons(a, m).subs(a, 1), Cons(a, m).subs(a, 1).free_symbols)

(Cons(1, m), {m})

In [22]:
# object that are substitued for should be SymPy expression
# the following works
print(srepr( Cons(a, m).subs(m, List(2, 3)) ))

assert Cons(a, m).subs(m, List(2, 3)) == List(a, 2, 3)

(Cons(a, m).subs(m, List(2,3)), Cons(a, m).subs(m,List(2,3)).free_symbols) 

List(Symbol('a'), Integer(2), Integer(3))


([a, 2, 3], {a})

In [23]:
# but it does not work now
# print(srepr( Cons(a, m).subs(m, [2, 3]) ))
# assert Cons(a, m).subs(m, [2, 3]) == List(a, 2, 3)
# (Cons(a, m).subs(m,[2,3]), Cons(a, m).subs(m,[2,3]).free_symbols) 

In [24]:
Cons(a, Cons(b, l))

Cons(a, Cons(b, l))

In [25]:
Cons(a, Cons(b, l)).doit()

Cons(a, Cons(b, l))

In [26]:
Cons(1, List(2, 3))

[1, 2, 3]

In [27]:
# now it works!
Cons(a, m).subs(a, 1).subs(m, List(2,3))

[1, 2, 3]

In [28]:
# simultaneous replacement
Cons(a, m).subs({a:1, m:List(2,3)}, simultaneous=True)

[1, 2, 3]

In [29]:
print(srepr(Cons(a, [2,3])))
print(srepr(Cons(1, m)))
print(srepr(Cons(1, [2,3])))

assert Cons(a, [2,3]) == List(a, 2, 3)
assert Cons(1, m) == Cons(1, m)
assert Cons(1, [2, 3]) == List(1, 2, 3)

(Cons(a, [2,3]), Cons(1,m), Cons(1, [2,3]))

List(Symbol('a'), Integer(2), Integer(3))
Cons(Integer(1), ListSymbol(Str('m')))
List(Integer(1), Integer(2), Integer(3))


([a, 2, 3], Cons(1, m), [1, 2, 3])

#### TODO

In [30]:
# TODO: use ListAdd instead of Add
print(srepr(Cons(a, m).doit()))
Cons(a, m).doit()

Cons(Symbol('a'), ListSymbol(Str('m')))


Cons(a, m)

In [31]:
# TODO: use ListAdd instead of Add
Cons(a, m).doit().subs(m, [2,3])

Cons(a, m)

### Len

In [32]:
S('len(l)', locals={'len': Len})

Len(l)

In [33]:
Len([])

0

In [34]:
(Len([1,2,3]), srepr(Len([1,2,3])))

(3, 'Integer(3)')

In [35]:
Len(l)

Len(l)

In [36]:
Len(l).kind

NumberKind

In [37]:
# we defined Len(x) as non-negative integer, so the following inequality is evaluated to true automatically.
Len(l) >= 0

True

In [38]:
Len(l) + 1

Len(l) + 1

In [39]:
Len(Cons(a, l))

Len(l) + 1

In [40]:
Len(Cons(a, Cons(b, l)))

Len(l) + 2

### IsNil

In [41]:
IsNil([])

True

In [42]:
IsNil(List())

True

In [43]:
IsNil([1])

False

In [44]:
IsNil(Cons(a, l))

False

In [45]:
IsNil(l)

IsNil(l)

In [46]:
IsNil(l).subs(l, List())

True

In [47]:
IsNil(l).subs(l, List(1,2))

False

----
### Map

In [48]:
Map(lambda x: x+1, List(1,2,3))

[2, 3, 4]

In [49]:
# fallback method: the previous notation is formal.
Map(lambda x: x+1, [1,2,3])

[2, 3, 4]

In [50]:
Map(S('lambda x: x+1'), l)

Map(Lambda(x, x + 1), l)

In [51]:
type(S('lambda x: x+1'))

sympy.core.function.Lambda

In [52]:
Map(S('lambda x: x+1'), l).subs(l, List(1,2,3))

[2, 3, 4]

In [53]:
Map(f, l)

Map(f, l)

In [54]:
Map(f, [1,2,3])

[f(1), f(2), f(3)]

In [55]:
srepr(Map(f, [1,2,3]))

"List(Function('f')(Integer(1)), Function('f')(Integer(2)), Function('f')(Integer(3)))"

In [56]:
Map(f, [1,2,3]).subs(f, S('lambda x: x+1'))

[2, 3, 4]

----
### All

In [57]:
All([True, False, True])

False

In [58]:
All([True, True, True])

True

In [59]:
All(l)

All(l)

In [60]:
All(l).subs(l, List(True, True, False))

False

In [61]:
All(l).subs(l, List(True, True, True))

True

----
### Map + All

In [62]:
All(Map(S('lambda x: x>0'), l))

All(Map(Lambda(x, x > 0), l))

In [63]:
All(Map(S('lambda x: x>0'), l)).subs(l, List(1,2,3))

True

In [64]:
All(Map(S('lambda x: x>0'), l)).subs(l, List(1,-1,3))

False

----
### TupleSymbol

In [65]:
Tuple(1,2,3)

(1, 2, 3)

In [66]:
Tuple(1,2,3) + Tuple(4,5)

(1, 2, 3, 4, 5)

In [67]:
Tuple(1,2,3)[1]

2

In [68]:
Tuple(a,b,a)[1]

b

In [69]:
t = TupleSymbol('t')
t

t

In [70]:
t[0]

GetItem(t, 0)

In [71]:
t[0].subs(t, Tuple(6,5,4))

6

----
### Lazy evaluation

In [72]:
f(a)

f(a)

In [73]:
srepr(f(a))

"Function('f')(Symbol('a'))"

In [74]:
f(a).subs(f, lambda x: x+1)

f(a)

In [75]:
f(a).subs(f, Lambda(a, a+1))

a + 1

In [76]:
f(a).subs(a, 1)

f(1)

In [77]:
# Note: ITE should specify boolean expressions in then and else clauses.
b1, b2 = symbols('b1 b2')
Lambda((), Piecewise((3, b1), (5, b2)))

     ⎧3  for b₁
() ↦ ⎨         
     ⎩5  for b₂

In [78]:
[rand_bool() for _ in range(5)]

[True, False, True, False, True]

In [79]:
Lambda((), Piecewise((3, b1), (5, b2))).subs({b1: rand_bool(), b2: rand_bool()})

() ↦ 3

In [80]:
[ Lambda((), Piecewise((3, b1), (5, b2))).subs({b1: rand_bool(), b2: rand_bool()})() for _ in range(5) ]

[5, 3, 3, 3, 5]

In [81]:
x1, z1, z2 = symbols('x1 z1 z2')
tc = Lambda((x1,), x1 > 0)
Lambda(x1, Piecewise((z1, b1 | tc(x1)), (S("fail"), True)))

     ⎧ z₁   for b₁ ∨ x₁ > 0
x₁ ↦ ⎨                     
     ⎩fail     otherwise   

In [82]:
Lambda(x1, Piecewise((z1, b1 | tc(x1)), (S("fail"), True))).subs(z1, rand_int())(3)

-93

In [83]:
Lambda(x1, Piecewise((z1, b1 | tc(x1)), (S("fail"), True))).subs({b1: rand_bool(), z1: rand_int()})(-1)

fail

In [84]:
h = Lambda(a, a+1)
h(f(20))

f(20) + 1

In [85]:
Lambda((a,), Subs(f(20), (f,), (a,)))

a ↦ (f(20))│   
           │f=a

In [89]:
f(y).subs(f, Lambda(x,x+1))

y + 1

----
### Inequality and Simplification

In [91]:
S('2*x+1 > 0')

2⋅x + 1 > 0

In [94]:
simplify(S('2*x+1 > 0'))

x > -1/2

In [103]:
e10 = S('sin(x)>1/3')
e10

sin(x) > 1/3

In [104]:
simplify(e10 & (x < 1))

        ⎛√2⎞        
x > atan⎜──⎟ ∧ x < 1
        ⎝4 ⎠        

In [105]:
simplify((Len(l) > 1) & (All(l)))

AttributeError: 'ListSymbol' object has no attribute 'binary_symbols'

----
## Matrix

Experiments on `MatrixSymbol` for study

In [ ]:
from sympy import MatrixSymbol, Identity
A = MatrixSymbol('A', 3, 3)
B = MatrixSymbol('B', 3, 3)                 

In [ ]:
A

In [ ]:
A+B

In [ ]:
repr(A)

In [ ]:
srepr(A)

In [ ]:
srepr(A+B)

In [ ]:
(A+B).free_symbols

In [ ]:
(A+B).subs(A, Identity(3))

In [ ]:
(A+B).subs(A, Identity(3)).subs(B, Identity(3))

In [ ]:
(A+B).subs(A, Identity(3)).subs(B, Identity(3)).doit()